In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
player_path = 'gdrive/My Drive/nba_summary_17_19.csv'
team_path = 'gdrive/My Drive/team_boxscores_2017_2018.csv'

In [0]:
import pandas as pd
player_df = pd.read_csv(player_path)
team_df = pd.read_csv(team_path)
print(player_df.columns, '\n\n',team_df.columns)

Index(['id', 'created_on', 'updated_on', 'source', 'is_duplicated',
       'display_name', 'match_id', 'team_id', 'team_abbr', 'team_city',
       'player_id', 'player_name', 'start_position', 'comment', 'min', 'fgm',
       'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct',
       'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts',
       'plus_minus', 'ast_pct', 'ast_ratio', 'ast_tov', 'blka', 'def_rating',
       'dreb_pct', 'e_def_rating', 'e_net_rating', 'e_off_rating', 'e_pace',
       'e_usg_pct', 'efg_pct', 'net_rating', 'off_rating',
       'opp_pts_2nd_chance', 'opp_pts_fb', 'opp_pts_off_tov', 'opp_pts_paint',
       'oreb_pct', 'pace', 'pfd', 'pie', 'pts_2nd_chance', 'pts_fb',
       'pts_off_tov', 'pts_paint', 'reb_pct', 'tm_tov_pct', 'ts_pct',
       'usg_pct', 'granularity', 'per_mode', 'season_id', 'dd2', 'fp', 'gp',
       'td3', 'hometeam_id', 'hometeam_abbr', 'visitorteam_id',
       'visitorteam_abbr', 'final_hscore', 'final_vscore',
     

In [0]:
# dataframe property
print(player_df.shape, team_df.shape)

(69863, 81) (5538, 62)


In [0]:
# filter match ids
match_id_field = list(player_df['match_id']) + list(team_df['match_id'])
unwanted_ids = [i for i in match_id_field if str(i)[0] != '2']
print("unwanted match_id number: ", len(unwanted_ids))

unwanted match_id number:  9665


In [0]:
# assumption testing -> match id is always 8 digits
match_id_len = [len(str(i)) == 8 for i in match_id_field]
print("Assumption is tested to be: ", all(match_id_len))

Assumption is tested to be:  True


In [0]:
# regular season match ids start with 2
# Filter all those starting with 2
def remove_non_nba(data):
  return data[(data['match_id'] >= 20000000) & (data['match_id'] < 30000000)]

In [0]:
# test match_id filter
nba_player_df = remove_non_nba(player_df)  # nba df is all match within nba(match id starts with 2)
nba_team_df = remove_non_nba(team_df)
print("Rows filtered out: ", (player_df.shape[0] + team_df.shape[0]) - (nba_player_df.shape[0] + nba_team_df.shape[0]))


Rows filtered out:  9665


In [0]:
# Now we try to understand the season data
season_id_field = list(set(player_df['season'])) + list(set(team_df['season_id']))
print("Season ids are: ", season_id_field)

Season ids are:  ['2017-18', '2018-19', 22017, 22018]


In [0]:
# convert season_id for consistency
pd.options.mode.chained_assignment = None

nba_player_df.loc[nba_player_df['season'] == '2018-19', 'season_id'] = 22018
nba_player_df.loc[nba_player_df['season'] == '2017-18', 'season_id'] = 22017
nba_player_df = nba_player_df.astype({'season_id':int})

In [0]:
# The data contains 2 seasons
# Separate 2017 and 2018 season
# Look at season_id attribute
def separate_season(data):
  return data[data['season_id'] == 22017], data[data['season_id'] == 22018]


In [0]:
# separate data based on season
player17_df, player18_df = separate_season(nba_player_df)
team17_df, team18_df = separate_season(nba_team_df)
print(player17_df.shape, player18_df.shape, team17_df.shape, team18_df.shape)

(30020, 81) (30796, 81) (2460, 62) (2460, 62)


In [0]:
# add a home/ away column to team df
def bool_home_away(player, team):
  # Use the players dataset
  # Compare team_id with hometeam_id
  # Then use match_ids in the two datasets to take home/away boolean
  # from players dataset to team dataset
  isHome = player['team_id'] == player['hometeam_id']
  player = player.assign(isHome = isHome.values)
  team = team.merge(player[['match_id', 'team_id', 'isHome']], on=['match_id', 'team_id'], how = 'left')
  team.drop_duplicates(inplace=True)
  return player, team

In [0]:
# add isHome to both player and team df
player17, team17 = bool_home_away(player17_df, team17_df)
player18, team18 = bool_home_away(player18_df, team18_df)
print(player17.shape, player18.shape, team17.shape, team18.shape)

(30020, 82) (30796, 82) (2460, 63) (2460, 63)


In [0]:
def pts_scored_recieved(player):
  player['pts_scored'] = 0 
  player['pts_received'] = 0
  player.loc[player['isHome'] == True, ['pts_scored']] = player['final_hscore'] # points scored = home score for home teams.
  player.loc[player['isHome'] == True, ['pts_received']] = player['final_vscore'] # points received = away score for home teams.
  player.loc[player['isHome'] == False, ['pts_scored']] = player['final_vscore']
  player.loc[player['isHome'] == False, ['pts_received']] = player['final_hscore']
  return player

In [0]:
pts_scored_recieved(player17)
pts_scored_recieved(player18)
print('Done')

Done


In [0]:
team18.head()

,match_id,team_id,team_name_x,team_abbr,team_city,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,to,pf,pts,plus_minus,source,team_name_y,e_off_rating,off_rating,e_def_rating,def_rating,e_net_rating,net_rating,ast_pct,ast_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,e_tm_tov_pct,tm_tov_pct,efg_pct,ts_pct,usg_pct,e_usg_pct,e_pace,pace,pie,team_name,pts_off_tov,pts_2nd_chance,pts_fb,pts_paint,opp_pts_off_tov,opp_pts_2nd_chance,opp_pts_fb,opp_pts_paint,blka,pfd,granularity,season_id,per_mode,isHome
0,21800729,1610612762,Jazz,UTA,Utah,240.0,37,94,0.394,13,40,0.325,19,25,0.760,16,31,47,25,9,8,11,19,106,4.0,https://stats.nba.com,Jazz,106.0,110.4,99.8,106.3,6.2,4.2,0.676,2.27,17.7,0.356,0.648,0.496,11.000,11.5,0.463,0.505,1.0,0.198,101.12,96.0,0.525,Jazz,12.0,12.0,8.0,38.0,13.0,19.0,12.0,48.0,5,22,team_game,22018,PerGame,True
13,21800729,1610612750,Timberwolves,MIN,Minnesota,240.0,38,90,0.422,8,26,0.308,18,21,0.857,12,38,50,20,5,5,14,22,102,-4.0,https://stats.nba.com,Timberwolves,99.8,106.3,106.0,110.4,-6.2,-4.2,0.526,1.33,14.9,0.352,0.644,0.504,14.671,15.6,0.467,0.514,1.0,0.196,101.12,96.0,0.475,Timberwolves,13.0,19.0,12.0,48.0,12.0,12.0,8.0,38.0,8,19,team_game,22018,PerGame,False
26,21800428,1610612758,Kings,SAC,Sacramento,240.0,45,96,0.469,15,44,0.341,20,25,0.800,7,35,42,33,6,3,13,28,125,-5.0,https://stats.nba.com,Kings,110.6,108.7,113.2,112.1,-2.6,-3.4,0.733,2.54,21.6,0.132,0.691,0.417,11.504,11.3,0.547,0.584,1.0,0.202,113.92,115.5,0.477,Kings,25.0,9.0,31.0,48.0,15.0,18.0,20.0,46.0,0,23,team_game,22018,PerGame,True
39,21800428,1610612744,Warriors,GSW,Golden State,240.0,44,96,0.458,15,40,0.375,27,36,0.750,15,45,60,32,5,0,18,23,130,5.0,https://stats.nba.com,Warriors,113.2,112.1,110.6,108.7,2.6,3.4,0.727,1.78,19.8,0.309,0.868,0.583,15.674,15.5,0.536,0.581,1.0,0.200,113.92,115.5,0.523,Warriors,15.0,18.0,20.0,46.0,25.0,9.0,31.0,48.0,3,28,team_game,22018,PerGame,False
50,21800081,1610612753,Magic,ORL,Orlando,240.0,33,101,0.327,10,43,0.233,15,17,0.882,11,30,41,21,9,6,11,22,91,-22.0,https://stats.nba.com,Magic,83.9,85.8,105.5,106.6,-21.6,-20.8,0.636,1.91,14.9,0.235,0.810,0.455,10.140,10.4,0.376,0.419,1.0,0.201,107.80,106.0,0.325,Magic,16.0,14.0,12.0,30.0,14.0,9.0,10.0,54.0,7,19,team_game,22018,PerGame,False


In [0]:
team17.head()


,match_id,team_id,team_name_x,team_abbr,team_city,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,to,pf,pts,plus_minus,source,team_name_y,e_off_rating,off_rating,e_def_rating,def_rating,e_net_rating,net_rating,ast_pct,ast_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,e_tm_tov_pct,tm_tov_pct,efg_pct,ts_pct,usg_pct,e_usg_pct,e_pace,pace,pie,team_name,pts_off_tov,pts_2nd_chance,pts_fb,pts_paint,opp_pts_off_tov,opp_pts_2nd_chance,opp_pts_fb,opp_pts_paint,blka,pfd,granularity,season_id,per_mode,isHome
0,21700366,1610612762,Jazz,UTA,Utah,240.0,34,77,0.442,11,30,0.367,22,27,0.815,4,28,32,17,10,5,13,21,101,-11.0,https://stats.nba.com,Jazz,103.2,105.2,111.4,114.3,-8.2,-9.1,0.500,1.31,14.3,0.130,0.696,0.413,13.282,13.5,0.513,0.568,1.0,0.200,99.22,97.0,0.446,Jazz,24.0,11.0,15.0,42.0,15.0,15.0,11.0,36.0,5,25,team_game,22017,PerGame,True
12,21700366,1610612745,Rockets,HOU,Houston,240.0,38,81,0.469,18,42,0.429,18,24,0.750,8,38,46,22,10,5,17,25,112,11.0,https://stats.nba.com,Rockets,111.4,114.3,103.2,105.2,8.2,9.1,0.579,1.29,16.9,0.304,0.870,0.587,16.905,17.3,0.580,0.612,1.0,0.199,99.22,97.0,0.554,Rockets,15.0,15.0,11.0,36.0,24.0,11.0,15.0,42.0,5,21,team_game,22017,PerGame,False
24,21700507,1610612754,Pacers,IND,Indiana,240.0,38,79,0.481,10,25,0.400,8,10,0.800,7,32,39,27,8,4,15,17,94,-4.0,https://stats.nba.com,Pacers,102.8,105.6,108.2,110.1,-5.3,-4.5,0.711,1.80,21.5,0.262,0.833,0.548,16.411,16.9,0.544,0.564,1.0,0.199,91.00,89.0,0.477,Pacers,8.0,8.0,3.0,34.0,18.0,4.0,15.0,46.0,3,12,team_game,22017,PerGame,True
36,21700507,1610612742,Mavericks,DAL,Dallas,240.0,40,78,0.513,8,29,0.276,10,15,0.667,4,31,35,26,7,3,9,12,98,4.0,https://stats.nba.com,Mavericks,108.2,110.1,102.8,105.6,5.3,4.5,0.650,2.60,21.6,0.167,0.738,0.452,11.038,11.2,0.564,0.579,1.0,0.197,91.00,89.0,0.523,Mavericks,18.0,4.0,15.0,46.0,8.0,8.0,3.0,34.0,4,17,team_game,22017,PerGame,False
47,21700105,1610612749,Bucks,MIL,Milwaukee,240.0,45,91,0.495,19,36,0.528,12,15,0.800,9,31,40,29,3,3,14,25,121,-5.0,https://stats.nba.com,Bucks,117.9,123.5,125.0,128.6,-7.1,-5.1,0.644,2.07,20.6,0.277,0.750,0.505,13.645,14.3,0.599,0.620,1.0,0.195,101.68,98.0,0.468,Bucks,11.0,17.0,13.0,40.0,19.0,15.0,5.0,52.0,3,18,team_game,22017,PerGame,False


In [0]:
def get_rolling_lr(data): # team data as input
  relevant_stats = ['match_id', 'team_id', 'isHome', 'plus_minus', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a',
                  'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 
                  'ast', 'stl', 'blk', 'to', 'pf', 'pts', 'e_off_rating', 
                  'off_rating', 'e_def_rating', 'def_rating', 'e_net_rating', 
                  'net_rating', 'ast_pct', 'ast_tov', 'ast_ratio', 'oreb_pct', 
                  'dreb_pct', 'reb_pct', 'e_tm_tov_pct', 'tm_tov_pct', 'efg_pct', 
                  'ts_pct', 'usg_pct', 'e_usg_pct', 'e_pace', 'pace', 'pie', 
                  'pts_off_tov', 'pts_2nd_chance', 'pts_fb', 'pts_paint', 'opp_pts_off_tov', 
                  'opp_pts_2nd_chance', 'opp_pts_fb', 'opp_pts_paint', 'blka', 'pfd']

  constant_attr=['match_id', 'team_id', 'isHome', 'plus_minus']
  
  data = data.sort_values('match_id', ascending=True) # sort by date (matchid is chronological)
  data.set_index(constant_attr,inplace=True) # set constants as index so they dont get averaged
  result= data.groupby("team_id").apply(lambda x:x.rolling(window=5).mean()) # get rolling avg
  dropped = result.dropna()
  result = dropped.groupby('match_id').apply(filter_single)
  result = result.reset_index(level=[1, 2, 3, 4]) # put indexes back in as columns
  result = result.drop(['match_id'], axis=1)
  result = result.reset_index(level=0)

  return result

In [0]:
def get_rolling(data):
  relevant_stats = ['match_id', 'team_id', 'isHome', 'pts', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a',
                  'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 
                  'ast', 'stl', 'blk', 'to', 'pf', 'e_off_rating', 
                  'off_rating', 'e_def_rating', 'def_rating', 'e_net_rating', 
                  'net_rating', 'ast_pct', 'ast_tov', 'ast_ratio', 'oreb_pct', 
                  'dreb_pct', 'reb_pct', 'e_tm_tov_pct', 'tm_tov_pct', 'efg_pct', 
                  'ts_pct', 'usg_pct', 'e_usg_pct', 'e_pace', 'pace', 'pie', 
                  'pts_off_tov', 'pts_2nd_chance', 'pts_fb', 'pts_paint', 'opp_pts_off_tov', 
                  'opp_pts_2nd_chance', 'opp_pts_fb', 'opp_pts_paint', 'blka', 'pfd']

  constant_attr=['match_id', 'team_id', 'isHome', 'pts']
  
  data = data.sort_values('match_id', ascending=True) # sort by date (matchid is chronological)
  data.set_index(constant_attr,inplace=True) # set constants as index so they dont get averaged
  result= data.groupby("team_id").apply(lambda x:x.rolling(window=5).mean()) # get rolling avg
  dropped = result.dropna()
  result = dropped.groupby('match_id').apply(filter_single)
  result = result.reset_index(level=[1, 2, 3, 4]) # put indexes back in as columns
  result = result.drop(['match_id'], axis=1)
  result = result.reset_index(level=0)

  return result

In [0]:
def filter_single(x):
  if  len(x)>1:
    return x

In [0]:
def get_logReg_test(teamdf): # testing dataset
  team = get_rolling(teamdf) # get moving avgs
  home = team.loc[team['isHome']==True]
  visitor = team.loc[team['isHome']==False] # divide as home- away
  visitor = visitor.rename(columns={'team_id': 'v_team_id', 'ast_tov' : 'v_ast_tov', 'oreb_pct': 'v_oreb_pct',
                                    'efg_pct': 'v_efg_pct', 'e_net_rating': 'v_net_rating'}) # identify visitor stats
  visitor = visitor[['match_id', 'v_ast_tov', 'v_oreb_pct', 'v_efg_pct', 'v_net_rating']]
  home = home[['match_id', 'ast_tov', 'oreb_pct', 'efg_pct', 'e_net_rating', 'plus_minus']]
  data = pd.merge(home, visitor, on='match_id') # merge home&visitor stats
  data['winner'] = 'N/A'
  data.loc[data['plus_minus']>0, ['winner']] = 1 #add winner column (1 if home, 0 if away)
  data.loc[data['plus_minus']<0, ['winner']] = 0
  data = data.drop(['plus_minus', 'match_id'], axis=1)
  return(data)

In [0]:
def get_logReg_train(teamdf): # training dataset
  team = teamdf
  home = team.loc[team['isHome']==True]
  visitor = team.loc[team['isHome']==False]
  visitor = visitor.rename(columns={'team_id': 'v_team_id', 'ast_tov' : 'v_ast_tov', 'oreb_pct': 'v_oreb_pct',
                                    'efg_pct': 'v_efg_pct', 'e_net_rating': 'v_net_rating'}) # identify visitor stats
  visitor = visitor[['match_id', 'v_ast_tov', 'v_oreb_pct', 'v_efg_pct', 'v_net_rating']]
  home = home[['match_id', 'ast_tov', 'oreb_pct', 'efg_pct', 'e_net_rating', 'plus_minus']]
  data = pd.merge(home, visitor, on='match_id') # merge home&visitor stats
  data['winner'] = 'N/A'
  data.loc[data['plus_minus']>0, ['winner']] = 1 #add winner column (1 if home, 0 if away)
  data.loc[data['plus_minus']<0, ['winner']] = 0
  data = data.drop(['plus_minus', 'match_id'], axis=1)
  return(data)

In [0]:
train = get_logReg_train(team17)
test = get_logReg_test(team18)

In [0]:
train.head()

,ast_tov,oreb_pct,efg_pct,e_net_rating,v_ast_tov,v_oreb_pct,v_efg_pct,v_net_rating,winner
0,1.31,0.130,0.513,-8.2,1.29,0.304,0.580,8.2,0
1,1.80,0.262,0.544,-5.3,2.60,0.167,0.564,5.3,0
2,1.91,0.250,0.619,7.1,2.07,0.277,0.599,-7.1,1
3,2.06,0.370,0.549,0.9,1.57,0.304,0.523,-0.9,1
4,1.76,0.364,0.520,19.3,0.64,0.160,0.494,-19.3,1


In [0]:
test.head()

,ast_tov,oreb_pct,efg_pct,e_net_rating,v_ast_tov,v_oreb_pct,v_efg_pct,v_net_rating,winner
0,1.824,0.2872,0.5686,11.74,1.666,0.3152,0.5184,-0.22,1
1,2.492,0.2666,0.5490,1.00,1.844,0.3100,0.5060,7.64,1
2,2.210,0.2918,0.5390,8.02,1.728,0.2362,0.5408,-10.56,1
3,2.092,0.2706,0.4742,-14.80,1.854,0.2246,0.6096,14.14,0
4,1.758,0.2990,0.5644,12.26,1.884,0.3000,0.5098,-8.66,1


In [0]:
from sklearn.linear_model import LogisticRegression

x_train, y_train = train.drop(columns=['winner']), train['winner'].to_numpy()

log_reg = LogisticRegression()
log_reg.fit(x_train, y_train) #train

x_test, y_test = test.drop(columns=['winner']), test['winner']

acc=log_reg.score(x_test, y_test) #test
print(acc)
# print("Baseline accuracy is {} on our test data".format(acc))

In [0]:
print(log_reg.coef_)
print(log_reg.intercept_)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class LargeNet(nn.Module):
    def __init__(self, num_inputs):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.fc1 = nn.Linear(num_inputs, 110)
        self.fc2 = nn.Linear(110, 22)
        self.fc3 = nn.Linear(22, 1)

    def forward(self, x, num_inputs):
        x = x.view(-1, num_inputs)
        x = self.fc1(x.float())
        x = F.relu(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [0]:
def getOffDef(df):
  team = get_rolling(df) # get moving avgs
  home = team.loc[team['isHome']==True]
  visitor = team.loc[team['isHome']==False] # divide as home- away
  # visitor = visitor.rename(columns={'team_id': 'v_team_id', 'ast_tov' : 'v_ast_tov', 'oreb_pct': 'v_oreb_pct',
                                    # 'efg_pct': 'v_efg_pct', 'e_net_rating': 'v_net_rating'}) # identify visitor stats
  visitorOff = visitor[['match_id', 'ast_tov', 'oreb_pct', 'efg_pct', 'e_net_rating', 'pts', 'isHome']]
  homeOff = home[['match_id', 'ast_tov', 'oreb_pct', 'efg_pct', 'e_net_rating', 'pts', 'isHome']]
  visitorDef = visitor[['match_id', 'stl', 'blk', 'e_def_rating', 'opp_pts_2nd_chance']]
  homeDef = home[['match_id', 'stl', 'blk', 'e_def_rating', 'opp_pts_2nd_chance']]
  data1 = pd.merge(homeOff, visitorDef, on='match_id') # merge home&visitor stats
  data2 = pd.merge(visitorOff, homeDef, on='match_id')
  # data1['pts_scored'] = 'N/A'
  # data2['pts_scored'] = 'N/A'
  # data1.loc[data1['match_id']==df['match_id'], ['pts_scored']] = df[['pts']]
  # data2.loc[data2['match_id']==df['match_id'], ['pts_scored']] = df[['final_vscore']]
  data = data1.append(data2)
  data = data.sort_values(['match_id', 'isHome'], ascending=True )
  return(data)

In [0]:
import torch
import numpy as np
from torch.utils import data

def get_loader(df):

  # df = df.sort_values('match_id', ascending=True)
  df = df.drop(['match_id'], axis=1)
  df_train, df_val, df_test = df.iloc[0:int(len(df)*0.7), :], df.iloc[int(len(df)*0.7):int(len(df)*0.85), :], df.iloc[int(len(df)*0.85):, :]

  train, test, val = get_xy(df_train), get_xy(df_test), get_xy(df_val)

  return train, test, val

def get_xy(df):

  df_y = df['pts'] # change to points scored by offensive team
  df_x = df.drop(['pts'], axis=1)
  np_x, np_y = list(df_x.to_numpy()), df_y.to_numpy()
  x, y = torch.tensor(np_x, dtype=float), torch.tensor(np_y, dtype=float)
  df = data.TensorDataset(x, y)
  dl = data.DataLoader(df, batch_size=1)
  return dl

In [0]:
def standardize(df, label):
    df = df.copy(deep=True)
    series = df.loc[:, label]
    avg = series.mean()
    stdv = series.std()
    series_standardized = (series - avg)/ stdv
    df.loc[:, label] = series_standardized
    return df

In [0]:
dataset = getOffDef(team17)
cols = ['ast_tov', 'oreb_pct', 'efg_pct', 'e_net_rating', 'stl', 'blk', 'e_def_rating', 'opp_pts_2nd_chance', 'pts']
dataset = standardize(dataset, cols)
dataset.head()

,match_id,ast_tov,oreb_pct,efg_pct,e_net_rating,pts,isHome,stl,blk,e_def_rating,opp_pts_2nd_chance
0,21700056,-0.753867,0.267881,-0.343474,-1.308448,-0.447994,False,0.600835,-1.970881,-0.653634,-1.268798
0,21700056,-0.160224,0.236107,-0.169069,0.432286,1.300252,True,-0.207125,-1.368683,1.422180,-0.377794
1,21700057,-0.981337,-0.341122,-0.030747,0.355148,-0.114995,False,0.600835,-0.465387,-0.366939,1.323215
1,21700057,-1.286481,0.702127,-0.992982,-1.069322,-0.198244,True,0.735495,0.437909,-0.610086,-0.944796
2,21700058,-0.726126,0.670353,0.474425,0.062026,0.051505,False,1.678116,-0.164288,0.580242,1.080213


In [0]:
import matplotlib.pyplot as plt

train, test, val = get_loader(dataset)

In [0]:
def training(model, loader, num_epochs = 15, num_inputs = 9):

  loss_fn = torch.nn.MSELoss(reduction='sum')
  learning_rate = 1e-3
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # x, y = [], []
  predictions, actual = [], []
  for i, data in enumerate(loader, 0): 
    inputs, labels = data
    y_pred = model(inputs, num_inputs)
    predictions.append(y_pred)
    actual.append(labels.float())
    loss = loss_fn(y_pred, labels.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return predictions, actual

In [0]:
def get_accuracy(model, a):
  pred, act = training(model, a)
  acc=0
  for x in range(len(pred)):
    if x%2==1:
      if (pred[x]>pred[x-1] and act[x]>act[x-1]) or (pred[x]<pred[x-1] and act[x]<act[x-1]):
        acc += 1
  return (2*acc/len(pred))

In [0]:
model = LargeNet(9)
get_accuracy(model, train)

In [0]:
model = LargeNet(9)
pred, act = training(model, test)

acc=0
for x in range(len(pred)):
  if x%2==1:
    if (pred[x]>pred[x-1] and act[x]>act[x-1]) or (pred[x]<pred[x-1] and act[x]<act[x-1]):
      acc += 1
print(2*acc/len(pred))

1633 1633
0.7054500918554807
